# ICO Datasets

In [23]:
import numpy as np
import pandas as pd
import altair as alt
import re
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce


# ICO Coffee Dataset for 2018
# Units: one-thousand 60-Kg bags
# Prices in USD per Kg

# year = '2018'

def process_data(year):
  '''
  Takes in many datasets and takes the relevant year and transforms the data into
  the desired qualities. The datasets are about the following:
  [consumption, exports, imports, prices paid to workers, and production]

  1) In order to run, simply take the file named Coffee Dataset and import all files within
  2) Then parameterize the function with your desired year

  Input: year --> (str)

  Output: df  --> (pd.DataFrame)
  '''

  # Domestic consumption by all exporting countries
  domestic_consumption = pd.read_csv('domestic-consumption.csv')[['domestic_consumption', year]]
  domestic_consumption = domestic_consumption.rename(columns={'domestic_consumption':'Country', year:'domestic_consumption'})
  domestic_consumption['domestic_consumption'] = domestic_consumption['domestic_consumption'].round(2)

  # Exports of all forms of coffee by all exporting countries
  exports_calendar_year = pd.read_csv('exports-calendar-year.csv')[['exports', year]]
  exports_calendar_year = exports_calendar_year.rename(columns={'exports':'Country', year:'exports'})
  exports_calendar_year['exports'] = exports_calendar_year['exports'].round(2)

  # Total exports by all exporting countries
  exports_crop_year = pd.read_csv('exports-crop-year.csv')[['exports_crop_year', year]]
  exports_crop_year = exports_crop_year.rename(columns={'exports_crop_year':'Country', year:'exports_crop_year'})
  exports_crop_year['exports_crop_year'] = exports_crop_year['exports_crop_year'].round(2)

  # Imports by selected importing countries
  imports = pd.read_csv('imports.csv')[['imports', year]]
  imports = imports.rename(columns={'imports': 'Country', year:'imports'})
  imports['imports'] = imports['imports'].round(2)

  # ICO Composite and group indicator prices
  prices_paid_to_growers = pd.read_csv('prices-paid-to-growers.csv')[['prices_paid_to_growers', year]]
  prices_paid_to_growers = prices_paid_to_growers.rename(columns={'prices_paid_to_growers':'Country', year:'prices_paid_to_growers'})
  prices_paid_to_growers['prices_paid_to_growers'] = prices_paid_to_growers['prices_paid_to_growers'].round(2)

  # Total production by all exporting countries
  total_production = pd.read_csv('total-production.csv')[['total_production', year]]
  total_production = total_production.rename(columns={'total_production':'Country', year:'total_production'})
  total_production['total_production'] = total_production['total_production'].round(2)

  # Gross opening stocks in all exporting countries
  #     This refers to the total amount of coffee available for exportation at the beginning
  #     of a specified period. It includes all coffee stocks held by exporting countries,
  #     which could be stored in warehouses, processing facilities, or any other storage locations.
  gross_opening_stocks = pd.read_csv('gross-opening-stocks.csv')[['gross_opening_stocks', year]]
  gross_opening_stocks = gross_opening_stocks.rename(columns={'gross_opening_stocks':'Country', year:'gross_opening_stocks'})

  datasets = [domestic_consumption,
              exports_calendar_year,
              exports_crop_year,
              imports,
              prices_paid_to_growers,
              total_production,
              gross_opening_stocks]

  # Adding in only the datasets that are of shape (56, 2)
  dfs_to_merge = []
  for df in datasets:
    if df.shape == (56, 2):
      dfs_to_merge.append(df)

  df = reduce(lambda left, right: pd.merge(left, right, on='Country'), dfs_to_merge)
  df.head()

  # Adding the country IDs
  codes = pd.read_csv('world-110m-country-names.txt', delimiter='\t')
  codes = codes.rename(columns={'name':'Country'})
  df = df.merge(codes, on='Country')

  # Adding longitude and latitude
  lat_lon = pd.read_csv('countries_lat_lon.txt', delimiter='\t')
  df = df.merge(lat_lon, on='Country')

  # # Adding world population and continent
  world_pop = pd.read_csv('world_population.csv')
  nineties = ['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
             '1998', '1999']
  two_thousands = ['2000', '2001', '2002', '2003', '2004', '2005',
              '2006', '2007', '2008', '2009']
  twenty_tens = ['2010', '2011', '2012', '2013',
              '2014', '2015', '2016', '2017', '2018']
  if year in nineties:
    world_pop['Avg Decade Population'] = (world_pop['1990 Population'] + world_pop['2000 Population']) / 2
  if year in two_thousands:
    world_pop['Avg Decade Population'] = (world_pop['2000 Population'] + world_pop['2010 Population']) / 2
  if year in twenty_tens:
    world_pop['Avg Decade Population'] = (world_pop['2010 Population'] + world_pop['2020 Population']) / 2
  world_pop = world_pop[['Country/Territory', 'Continent', 'Avg Decade Population', 'Density (per km²)']]
  world_pop = world_pop.rename(columns={'Country/Territory':'Country'})
  df = df.merge(world_pop, on='Country')

  df = df.rename(columns = {'domestic_consumption':'Domestic Consumption',
                            'exports_crop_year':'Exports Crop Year',
                            'total_production':'Total Production',
                            'gross_opening_stocks':'Gross Opening Stocks',
                            'Density (per km²)':'Density'})

  return df

In [24]:
# Puts all the years dataframes into a dictionary that can be easily traversed
# through and accessed for time series analysis
all_years = ['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
             '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
             '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
             '2014', '2015', '2016', '2017', '2018']

all_sets = {}

for year in all_years:
  all_sets[f'ico_df_{year}'] = process_data(year)
  all_sets[f'ico_df_{year}']['Year'] = int(year)

ico_all_years = pd.concat(all_sets.values(), ignore_index=True)
ico_all_years.shape

(1363, 14)

In [25]:
# Create csv for download to local machine
# ico_df_2018.to_csv('ico_df_2018.csv')

# Visualized 2018 DataFrame
ico_df_2018 = process_data('2018')
ico_df_2018.head()

,Country,Domestic Consumption,exports,Exports Crop Year,Total Production,Gross Opening Stocks,id,Name,latitude,longitude,Continent,Avg Decade Population,Density
0,Angola,30.0,9.32,10.39,40.39,5.0,24,AO,-11.202692,17.873887,Africa,28396335.0,28.5466
1,Brazil,22250.0,35382.56,37613.88,62924.88,9832.0,76,BR,-14.235004,-51.925280,South America,204774898.0,25.2841
2,Burundi,2.0,201.73,179.92,178.42,10.0,108,BI,-3.373056,29.918886,Africa,10673416.0,463.0874
3,Ecuador,155.0,482.70,446.00,601.00,3.0,218,EC,-1.831239,-78.183406,South America,16289090.0,65.0229
4,Indonesia,4800.0,4538.84,4717.59,9417.59,400.0,360,ID,-0.789275,113.921327,Asia,257937071.5,144.6529


In [27]:
# Select the hour column to slide on
years= alt.selection_single(
    fields=['Year'],
    bind=alt.binding_range(min=1990, max=2018, step=1, name='Year'),
    init={'Year': 1990}
)

# Define population
population = ico_all_years['Avg Decade Population']

# Normalize Total Production and Domestic Consumption
ico_all_years['normalized_production'] = (ico_all_years['Total Production'] / population) * 1000000
max_normalized_production = ico_all_years['normalized_production'].max()
ico_all_years['normalized_consumption'] = (ico_all_years['Domestic Consumption'] / population) * 1000000
max_normalized_consumption = ico_all_years['normalized_consumption'].max()

# Production vs Population
production = alt.Chart(ico_all_years).mark_circle(size=40).encode(
    x=alt.X('normalized_production:Q',
            title='Domestic Consumption',
            scale=alt.Scale(domain=(-5, max_normalized_consumption))),
    y=alt.Y('Density:Q',
            title='Population Density',
            scale=alt.Scale(domain=(-5, max_normalized_production))),
    color=alt.Color('Continent:N',
                    scale=alt.Scale(scheme='category10')),
    tooltip=['Country',
             'Continent',
             'Total Production',
             'Domestic Consumption']
    ).properties(
        width=500,
        height=300,
        title={
            'text':'Normalized Population: Total Production vs Population Density',
            'subtitle':'Coffee Production per 1,000,000 People'}
    ).add_selection(
        years
    ).transform_filter(
        years
    ).interactive()

# Consumption vs Population
consumption = alt.Chart(ico_all_years).mark_circle(size=40).encode(
    x=alt.X('normalized_consumption:Q',
            title='Domestic Production',
            scale=alt.Scale(domain=(-5, max_normalized_consumption))),
    y=alt.Y('Density:Q',
            title='Population Density',
            scale=alt.Scale(domain=(-5, max_normalized_production))),
    color=alt.Color('Continent:N',
                    scale=alt.Scale(scheme='category10')),
    tooltip=['Country',
             'Continent',
             'Total Production',
             'Domestic Consumption']
    ).properties(
        width=500,
        height=300,
        title={
            'text':'Normalized Population: Domestic Consupmtion vs Population Density',
            'subtitle':'Coffee Production per 1,000,000 People'}
    ).add_selection(
        years
    ).transform_filter(
        years
    ).interactive()

production | consumption

In [ ]:
# Create dataframe by continent
africa = ico_all_years[ico_all_years['Continent'] == 'Africa']
asia = ico_all_years[ico_all_years['Continent'] == 'Asia']
northAmerica = ico_all_years[ico_all_years['Continent'] == 'North America']
oceania = ico_all_years[ico_all_years['Continent'] == 'Oceania']
southAmerica = ico_all_years[ico_all_years['Continent'] == 'South America']

In [ ]:
# URL for all flags in ico_all_years dataframe
africa_flags = {
    'Angola': 'https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg',
    'Burundi': 'https://upload.wikimedia.org/wikipedia/commons/5/50/Flag_of_Burundi.svg',
    'Madagascar': 'https://upload.wikimedia.org/wikipedia/commons/b/bc/Flag_of_Madagascar.svg',
    'Malawi': 'https://upload.wikimedia.org/wikipedia/commons/d/d1/Flag_of_Malawi.svg',
    'Rwanda': 'https://upload.wikimedia.org/wikipedia/commons/1/17/Flag_of_Rwanda.svg',
    'Zimbabwe': 'https://upload.wikimedia.org/wikipedia/commons/6/6a/Flag_of_Zimbabwe.svg',
    'Zambia': 'https://upload.wikimedia.org/wikipedia/commons/0/06/Flag_of_Zambia.svg',
    'Benin': 'https://upload.wikimedia.org/wikipedia/commons/0/0a/Flag_of_Benin.svg',
    'Cameroon': 'https://upload.wikimedia.org/wikipedia/commons/4/4f/Flag_of_Cameroon.svg',
    'Central African Republic': 'https://upload.wikimedia.org/wikipedia/commons/6/6f/Flag_of_the_Central_African_Republic.svg',
    'Equatorial Guinea': 'https://upload.wikimedia.org/wikipedia/commons/3/31/Flag_of_Equatorial_Guinea.svg',
    'Ethiopia': 'https://upload.wikimedia.org/wikipedia/commons/7/71/Flag_of_Ethiopia.svg',
    'Gabon': 'https://upload.wikimedia.org/wikipedia/commons/0/04/Flag_of_Gabon.svg',
    'Ghana': 'https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Ghana.svg',
    'Guinea': 'https://upload.wikimedia.org/wikipedia/commons/e/ed/Flag_of_Guinea.svg',
    'Kenya': 'https://upload.wikimedia.org/wikipedia/commons/4/49/Flag_of_Kenya.svg',
    'Liberia': 'https://upload.wikimedia.org/wikipedia/commons/b/b8/Flag_of_Liberia.svg',
    'Nigeria': 'https://upload.wikimedia.org/wikipedia/commons/7/79/Flag_of_Nigeria.svg',
    'Sierra Leone': 'https://upload.wikimedia.org/wikipedia/commons/1/17/Flag_of_Sierra_Leone.svg',
    'Togo': 'https://upload.wikimedia.org/wikipedia/commons/6/68/Flag_of_Togo.svg',
    'Uganda': 'https://upload.wikimedia.org/wikipedia/commons/4/4e/Flag_of_Uganda.svg'
}

asia_flags = {
    'Indonesia': 'https://upload.wikimedia.org/wikipedia/commons/9/9f/Flag_of_Indonesia.svg',
    'Timor-Leste': 'https://upload.wikimedia.org/wikipedia/commons/2/26/Flag_of_East_Timor.svg',
    'Philippines': 'https://upload.wikimedia.org/wikipedia/commons/9/99/Flag_of_the_Philippines.svg',
    'India': 'https://upload.wikimedia.org/wikipedia/en/4/41/Flag_of_India.svg',
    'Nepal': 'https://upload.wikimedia.org/wikipedia/commons/9/9b/Flag_of_Nepal.svg',
    'Sri Lanka': 'https://upload.wikimedia.org/wikipedia/commons/1/11/Flag_of_Sri_Lanka.svg',
    'Thailand': 'https://upload.wikimedia.org/wikipedia/commons/a/a9/Flag_of_Thailand.svg',
    'Yemen': 'https://upload.wikimedia.org/wikipedia/commons/8/89/Flag_of_Yemen.svg'
}

northAmerica_flags = {
    'Cuba': 'https://upload.wikimedia.org/wikipedia/commons/b/bd/Flag_of_Cuba.svg',
    'Dominican Republic': 'https://upload.wikimedia.org/wikipedia/commons/9/9f/Flag_of_the_Dominican_Republic.svg',
    'Haiti': 'https://upload.wikimedia.org/wikipedia/commons/5/56/Flag_of_Haiti.svg',
    'Costa Rica': 'https://upload.wikimedia.org/wikipedia/commons/b/bc/Flag_of_Costa_Rica.svg',
    'El Salvador': 'https://upload.wikimedia.org/wikipedia/commons/3/34/Flag_of_El_Salvador.svg',
    'Guatemala': 'https://upload.wikimedia.org/wikipedia/commons/e/ec/Flag_of_Guatemala.svg',
    'Honduras': 'https://upload.wikimedia.org/wikipedia/commons/8/82/Flag_of_Honduras.svg',
    'Jamaica': 'https://upload.wikimedia.org/wikipedia/commons/0/0a/Flag_of_Jamaica.svg',
    'Mexico': 'https://upload.wikimedia.org/wikipedia/commons/f/fc/Flag_of_Mexico.svg',
    'Nicaragua': 'https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Nicaragua.svg',
    'Panama': 'https://upload.wikimedia.org/wikipedia/commons/a/ab/Flag_of_Panama.svg'
}

oceania_flags = {
    'Papua New Guinea': 'https://upload.wikimedia.org/wikipedia/commons/e/e3/Flag_of_Papua_New_Guinea.svg'
}

southAmerica_flags = {
    'Brazil': 'https://upload.wikimedia.org/wikipedia/commons/0/05/Flag_of_Brazil.svg',
    'Ecuador': 'https://upload.wikimedia.org/wikipedia/commons/e/e8/Flag_of_Ecuador.svg',
    'Paraguay': 'https://upload.wikimedia.org/wikipedia/commons/2/27/Flag_of_Paraguay.svg',
    'Peru': 'https://upload.wikimedia.org/wikipedia/commons/c/cf/Flag_of_Peru.svg',
    'Colombia': 'https://upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Colombia.svg',
    'Guyana': 'https://upload.wikimedia.org/wikipedia/commons/9/99/Flag_of_Guyana.svg'
}

In [ ]:
# Function to create a by year consumer / production slide chart per continent
def cons_prod_chart(continent, flags): 
  # Select the hour column to slide on
  years = alt.selection_single(
      fields=['Year'],
      bind=alt.binding_range(min=1990, max=2018, step=1, name='Year'),
      init={'Year': 1990}
  )

  # Define population
  population = continent['Avg Decade Population']

  # adding in flag image
  continent['flag_url'] = continent['Country'].map(flags)
  
  # Normalize Total Production and Domestic Consumption
  continent['normalized_production'] = (continent['Total Production'] / population) * 1000000
  max_normalized_production = continent['normalized_production'].max()
  continent['normalized_consumption'] = (continent['Domestic Consumption'] / population) * 1000000
  max_normalized_consumption = continent['normalized_consumption'].max()
  max_population = continent['Density'].max()


  # Production vs Population
  production = alt.Chart(continent).mark_image(
      width=30,
      height=30
      ).encode(
      x=alt.X('normalized_production:Q',
              title='Total Production',
              scale=alt.Scale(domain=(-5, max_normalized_production))),
      y=alt.Y('Density:Q',
              title='Population Density',
              scale=alt.Scale(domain=(-5, max_population))),
      url='flag_url',
      tooltip=['Country',
              'Continent',
              'Total Production',
              'Domestic Consumption']
      ).properties(
          width=500,
          height=300,
          title={
              'text':'Normalized Population: Total Production vs Population Density',
              'subtitle':'Coffee Production per 1,000,000 People'}
      ).add_selection(
          years
      ).transform_filter(
          years
      ).interactive()


  # Consumption vs Population
  consumption = alt.Chart(continent).mark_image(
      width=30,
      height=30
  ).encode(
      x=alt.X('normalized_consumption:Q',
              title='Domestic Consumption',
              scale=alt.Scale(domain=(-5, max_normalized_consumption))),
      y=alt.Y('Density:Q',
              title='Population Density',
              scale=alt.Scale(domain=(-5, max_population))),
      url='flag_url',
      tooltip=['Country',
              'Continent',
              'Total Production',
              'Domestic Consumption']
      ).properties(
          width=500,
          height=300,
          title={
              'text':'Normalized Population: Domestic Consupmtion vs Population Density',
              'subtitle':'Coffee Consumption per 1,000,000 People'}
      ).add_selection(
          years
      ).transform_filter(
          years
      ).interactive()

  return production | consumption

In [ ]:
# Africa chart
cons_prod_chart(africa, africa_flags)

In [ ]:
# Asia chart
cons_prod_chart(asia, asia_flags)

In [ ]:
# North America chart
cons_prod_chart(northAmerica, northAmerica_flags)

In [ ]:
# Oceania chart
cons_prod_chart(oceania, oceania_flags)

In [ ]:
# South America chart
cons_prod_chart(southAmerica, southAmerica_flags)